In [1]:
import datasets
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, DataCollator, TrainingArguments, Trainer
from dataclasses import dataclass, field

import torch

import random
import pandas as pd
import numpy as np

In [2]:
dataset = load_dataset('amazon_reviews_multi')

No config specified, defaulting to: amazon_reviews_multi/all_languages
Reusing dataset amazon_reviews_multi (C:\Users\chris\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


In [3]:
dataset["train"][0:5]

{'review_id': ['de_0203609',
  'de_0559494',
  'de_0238777',
  'de_0477884',
  'de_0270868'],
 'product_id': ['product_de_0865382',
  'product_de_0678997',
  'product_de_0372235',
  'product_de_0719501',
  'product_de_0022613'],
 'reviewer_id': ['reviewer_de_0267719',
  'reviewer_de_0783625',
  'reviewer_de_0911426',
  'reviewer_de_0836478',
  'reviewer_de_0736276'],
 'stars': [1, 1, 1, 1, 1],
 'review_body': ['Armband ist leider nach 1 Jahr kaputt gegangen',
  'In der Lieferung war nur Ein Akku!',
  'Ein Stern, weil gar keine geht nicht. Es handelt sich um gebraucht Waren, die Stein haben so ein Belag drauf, wo man sich dabei denken kann, dass jemand schon die benutzt und nicht Mal richtig gewaschen. Bei ein paar ist die Qualität Mangelhaft, siehe Bild. Ein habe ich ausprobiert, richtig gewaschen, dann verfärbt sich..... Wärme halt nicht lange. Deswegen wird es zurückgeschickt.',
  'Dachte, das wären einfach etwas festere Binden, vielleicht größere Always. Aber die Verpackung ist dera

In [4]:
metric = load_metric('accuracy')
f1_metric = load_metric('f1')

In [15]:
do_shard=True
if do_shard:
    dataset = dataset.shuffle(seed=8855)
    train_dataset=dataset["train"].shard(index=1, num_shards=10)
    val_dataset=dataset["validation"].shard(index=1, num_shards=5)
else:
    train_dataset=dataset["train"]
    val_dataset=dataset["validation"]

Loading cached shuffled indices for dataset at C:\Users\chris\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-ee52c5ebc979ea41.arrow
Loading cached shuffled indices for dataset at C:\Users\chris\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-88d6095512481bd8.arrow
Loading cached shuffled indices for dataset at C:\Users\chris\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-c59e3d568381596d.arrow


In [16]:
model_checkpoint='./model/checkpoint-90000'
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [17]:
pad_to_max=False
def tokenize_data(example):
    text_ = example["review_body"] + " " + example["review_title"] + " " + example["product_category"]
    encodings = tokenizer(text_, pad_to_max_length=pad_to_max,
                                truncation=True,
                                add_special_tokens=True,
                                return_token_type_ids=False,
                                return_attention_mask=True,
                                return_overflowing_tokens=False,
                                return_special_tokens_mask=False,
                                )
    encodings["labels"] = example["stars"] - 1
    return encodings

In [18]:
encoded_train_dataset = train_dataset.map(tokenize_data)
encoded_val_dataset = val_dataset.map(tokenize_data)

100%|██████████| 6000/6000 [00:02<00:00, 2756.09ex/s]


In [21]:
def pad_seq(seq, max_batch_len, pad_value):
    return seq + (max_batch_len - len(seq)) * [pad_value]

In [22]:
@dataclass
class SmartCollator():
    pad_token_id: int

    def __call__(self, batch):
        batch_inputs = list()
        batch_attention_mask = list()
        labels = list()
        max_size = max(len(ex['input_ids']) for ex in batch)
        for item in batch:
            batch_inputs += [pad_seq(item['input_ids'], max_size, self.pad_token_id)]
            batch_attention_mask += [pad_seq(item['attention_mask'], max_size, 0)]
            labels.append(item['labels'])
        
        return {"input_ids": torch.tensor(batch_inputs, dtype=torch.long),
                "attention_mask": torch.tensor(batch_attention_mask, dtype=torch.long),
                "labels": torch.tensor(labels, dtype=torch.long)
                }

In [5]:
BATCH_SIZE = 4
NUM_LABELS = 5

resume_training = True
if resume_training:
    model_checkpoint = './model/checkpoint-90000'
else:
    model_checkpoint = 'xlm-roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

In [7]:
METRIC_NAME = 'accuracy'

args = TrainingArguments(
    output_dir='./model',
    seed=8855,
    evaluation_strategy='steps',
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_NAME,
    eval_steps=5000,
    save_steps=5000,
    fp16=True,
)

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return metric.compute(predictions=predictions, references=labels)

In [14]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    data_collator=SmartCollator(pad_token_id=tokenizer.pad_token_id),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using amp half precision backend


In [15]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: product_id, stars, reviewer_id, review_id, language, review_body, review_title, product_category.
***** Running training *****
  Num examples = 120000
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 90000
  1%|          | 501/90000 [00:56<2:50:51,  8.73it/s]

{'loss': 1.6121, 'learning_rate': 1.988911111111111e-05, 'epoch': 0.02}


  1%|          | 1001/90000 [01:52<2:51:21,  8.66it/s]

{'loss': 1.619, 'learning_rate': 1.9778000000000003e-05, 'epoch': 0.03}


  2%|▏         | 1501/90000 [02:48<2:47:11,  8.82it/s]

...

                                                       
100%|██████████| 90000/90000 [2:45:52<00:00, 10.13it/s]Saving model checkpoint to ./model\checkpoint-90000
Configuration saved in ./model\checkpoint-90000\config.json


{'eval_loss': 1.6094287633895874, 'eval_accuracy': 0.20083333333333334, 'eval_runtime': 29.3324, 'eval_samples_per_second': 204.552, 'eval_steps_per_second': 51.138, 'epoch': 3.0}


Model weights saved in ./model\checkpoint-90000\pytorch_model.bin
tokenizer config file saved in ./model\checkpoint-90000\tokenizer_config.json
Special tokens file saved in ./model\checkpoint-90000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./model\checkpoint-30000 (score: 0.203).
Could not locate the best model at ./model\checkpoint-30000\pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.
100%|██████████| 90000/90000 [2:46:26<00:00,  9.01it/s]

{'train_runtime': 9986.9427, 'train_samples_per_second': 36.047, 'train_steps_per_second': 9.012, 'train_loss': 1.6123690273708768, 'epoch': 3.0}


TrainOutput(global_step=90000, training_loss=1.6123690273708768, metrics={'train_runtime': 9986.9427, 'train_samples_per_second': 36.047, 'train_steps_per_second': 9.012, 'train_loss': 1.6123690273708768, 'epoch': 3.0})

In [16]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: product_id, stars, reviewer_id, review_id, language, review_body, review_title, product_category.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 4
100%|██████████| 1500/1500 [00:29<00:00, 50.91it/s]


{'eval_loss': 1.6094287633895874,
 'eval_accuracy': 0.20083333333333334,
 'eval_runtime': 29.7216,
 'eval_samples_per_second': 201.874,
 'eval_steps_per_second': 50.468,
 'epoch': 3.0}

In [9]:
args = TrainingArguments(
    output_dir='./test_results',
    seed=8855,
    evaluation_strategy='steps',
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_NAME,
    eval_steps=5000,
    save_steps=5000,
    fp16=True,
)

In [10]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

In [23]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    data_collator=SmartCollator(pad_token_id=tokenizer.pad_token_id),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

loading configuration file ./model/checkpoint-90000\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./model/checkpoint-90000",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "t

In [24]:
best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize")

[I 2022-01-31 16:42:33,524] A new study created in memory with name: no-name-da268278-32a6-4f68-834b-c3e3fca1b29c
Trial:
loading configuration file ./model/checkpoint-90000\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./model/checkpoint-90000",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
 

RuntimeError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 10.00 GiB total capacity; 8.26 GiB already allocated; 0 bytes free; 8.47 GiB reserved in total by PyTorch)

In [ ]:
best_run

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()